# Get numbers

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import matplotlib.ticker as ticker

import cptac
import cptac.utils as u

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
df_fdr_pvals = pd.read_csv(root+R"\all_pval_all_proteins.csv")
df_fdr_pvals = df_fdr_pvals.loc[df_fdr_pvals['Proteomics'] != 'PTEN'] # Drop PTEN

# Drop Kidney
cols = []
df_fdr_pvals = df_fdr_pvals.drop(cols)
df_fdr_pvals
len(df_fdr_pvals)


15097

In [3]:
def Pvalue_sig(row):
    numSig = 0

    for item in row:
        if pd.isnull(item):
            continue
        if item < 0.05: # changed from .1????
            numSig += 1
            
    return numSig

In [4]:
#Only include genes that have data for at least 6/8 genes.
#df_fdr_pvals = df_fdr_pvals.dropna(thresh=12, axis=0)

In [5]:
df_fdr_pvals["Num_sig"] = df_fdr_pvals.loc[:,["Gbm_P_Value","Hnscc_P_Value","Luad_P_Value","Lscc_P_Value", "Brca_P_Value","Ov_P_Value","En_P_Value","Colon_P_Value"]].apply(Pvalue_sig, axis = 1)
df_fdr_pvals = df_fdr_pvals.sort_values(by=['Num_sig'],ascending=False)
df_fdr_pvals

,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Lscc_P_Value,Lscc_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median,Num_sig
0,ARMH3,5.766739e-07,-0.405134,0.024047,-0.135941,0.222235,-0.37845,0.906857,-0.27970,NaN,NaN,NaN,NaN,0.034811,0.1503,NaN,NaN,3
1799,IMP3,4.509668e-02,0.180037,0.017960,0.335729,0.834222,-0.15000,0.999848,0.14530,0.477786,0.36940,0.985704,-0.083353,0.028419,-0.3025,0.987216,-0.09850,3
1229,INTS3,2.456890e-02,0.243546,0.019886,0.184601,NaN,NaN,0.999848,0.03515,0.986192,0.00910,0.985704,-0.186980,0.025383,-0.2298,0.956095,0.05450,3
1543,IMP4,3.547217e-02,0.206208,0.014203,0.258588,0.751407,0.09615,0.999848,0.06315,0.484999,0.27270,0.985704,-0.108442,0.006415,-0.4177,0.926083,0.00900,3
455,XPO1,5.969823e-03,0.280093,0.003209,0.290587,NaN,NaN,0.999848,0.11865,0.848027,0.13985,0.985704,-0.035047,0.019773,-0.1533,0.990629,-0.00250,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,C8orf59,4.601737e-01,0.124998,0.543541,-0.118237,0.877141,0.09490,0.999848,0.28980,0.741807,0.36450,0.991931,-0.062953,0.682650,-0.1230,NaN,0.15150,0
6532,SLC39A1,4.602313e-01,-0.306619,NaN,0.230950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6534,ERBB3,4.602513e-01,-0.185341,0.792957,-0.127523,0.544745,-0.44345,0.999848,-0.13005,0.727816,-0.21755,0.985704,-0.350727,0.108702,-0.2850,0.917096,-0.00195,0
6535,SLC25A52,4.605134e-01,-0.167080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.985704,-0.198085,NaN,NaN,NaN,NaN,0


In [6]:
#Keep trans genes that are significant in atleast 1 cancer. 

df_fdr_pvals_1 = df_fdr_pvals[df_fdr_pvals['Num_sig'].isin([1,2,3,4,5,6,7,8])]
len(df_fdr_pvals_1.Proteomics.unique())

2630

In [7]:
#Keep trans genes that are significant in 1 cancer. 

one = df_fdr_pvals[df_fdr_pvals['Num_sig'].isin([1])]
len(one.Proteomics.unique())

2298

In [8]:
#Keep trans genes that are significant in mult cancer. 

mult = df_fdr_pvals[df_fdr_pvals['Num_sig'].isin([2,3,4,5,6,7,8])]
len(mult.Proteomics.unique())

332

In [9]:
total_sig = len(df_fdr_pvals_1)
sig_one = df_fdr_pvals[df_fdr_pvals['Num_sig'].isin([1])]
mult = df_fdr_pvals[df_fdr_pvals['Num_sig'].isin([2,3])]
m = len(mult)
one = len(sig_one)
print(total_sig, '-', one, '=', int(total_sig)-int(one))
print(one, '/', total_sig, '=', int(one)/int(total_sig))
print(m, '/', total_sig, '=', int(m)/int(total_sig))

2630 - 2298 = 332
2298 / 2630 = 0.8737642585551331
332 / 2630 = 0.12623574144486693


In [ ]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False
    for item in row: 
        if pd.isnull(item):
            continue
        if item < 0:
            hasNeg = True
        if item > 0:
            hasPos = True
           
    if hasPos & hasNeg:
        return True
    return False

In [ ]:
only_med = df_fdr_pvals_1.drop(columns= ['Gbm_P_Value','Hnscc_P_Value','Luad_P_Value','Lscc_P_Value',
                     'Brca_P_Value','Ov_P_Value','En_P_Value','Colon_P_Value'])
only_med